In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
fpath = "/nfs/turbo/umms-indikar/shared/projects/hybrid_reprogramming/data/adapter_search/seqkit.locate.csv"
df = pd.read_csv(fpath, sep='\t')
print(f"{df.shape=}")
df.head()

df.shape=(70470920, 7)


,seqID,patternName,pattern,strand,start,end,matched
0,be606770-65a1-40be-89a6-c7056ce4d70e,polyT,TTTTTTTTTTT,+,1,11,TTTTTTTTTTT
1,be606770-65a1-40be-89a6-c7056ce4d70e,polyT,TTTTTTTTTTT,+,2,12,TTTTTTTTTTT
2,be606770-65a1-40be-89a6-c7056ce4d70e,polyT,TTTTTTTTTTT,+,3,13,TTTTTTTTTTT
3,be606770-65a1-40be-89a6-c7056ce4d70e,polyT,TTTTTTTTTTT,+,4,14,TTTTTTTTTTT
4,be606770-65a1-40be-89a6-c7056ce4d70e,polyT,TTTTTTTTTTT,+,5,15,TTTTTTTTTTT


# How may have multiple GEM adapaters?

In [3]:
df['patternName'].unique()

array(['polyT', 'polyA', '10x_left_adapter', '10x_right_adapter',
       'biolegend_3', 'biolegend_2', 'biosig', 'biolegend_1'],
      dtype=object)

In [4]:
gdf = df[df['patternName'].isin(['10x_left_adapter', '10x_right_adapter',])]
gdf['n_matches'] = gdf.groupby('seqID')['patternName'].transform('count')
print(f"{gdf.shape=}")
gdf.head()

gdf.shape=(1770674, 8)


/tmp/ipykernel_3739695/1166519112.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdf['n_matches'] = gdf.groupby('seqID')['patternName'].transform('count')


,seqID,patternName,pattern,strand,start,end,matched,n_matches
516,325713bd-146b-4e0e-a288-27990b7e8111,10x_left_adapter,CTACACGACGCTCTTCCGATCT,-,450,471,CTACACGACGCTCTTCCGATCT,2
517,325713bd-146b-4e0e-a288-27990b7e8111,10x_right_adapter,CCCATGTACTCTGCGTTGATACCACTGCTT,-,83,112,CCCATGTACTCTGCGTTGATACCACTGCTT,2
806,bea48768-b89e-4e8a-8d4b-2cd2699056b1,10x_left_adapter,CTACACGACGCTCTTCCGATCT,-,717,738,CTACACGACGCTCTTCCGATCT,2
807,bea48768-b89e-4e8a-8d4b-2cd2699056b1,10x_right_adapter,CCCATGTACTCTGCGTTGATACCACTGCTT,-,256,285,CCCATGTACTCTGCGTTGATACCACTGCTT,2
862,cdefd141-2362-4f44-bfc6-6f99c4855b07,10x_left_adapter,CTACACGACGCTCTTCCGATCT,-,496,517,CTACACGACGCTCTTCCGATCT,2


In [5]:
gdf['n_matches'].value_counts(normalize=True)

n_matches
2     0.541711
3     0.252319
1     0.161231
4     0.044114
5     0.000274
6     0.000132
8     0.000032
7     0.000032
9     0.000030
17    0.000019
11    0.000019
15    0.000017
14    0.000016
27    0.000015
13    0.000015
12    0.000014
19    0.000011
Name: proportion, dtype: float64

# summary

In [6]:
df = df.drop_duplicates(subset=['seqID', 'patternName', 'strand'])
df['n_matches'] = df.groupby('seqID')['patternName'].transform('nunique')
print(f"{df.shape=}")
df.head()

df.shape=(4916499, 8)


,seqID,patternName,pattern,strand,start,end,matched,n_matches
0,be606770-65a1-40be-89a6-c7056ce4d70e,polyT,TTTTTTTTTTT,+,1,11,TTTTTTTTTTT,2
236,be606770-65a1-40be-89a6-c7056ce4d70e,polyA,AAAAAAAAAAA,-,236,246,AAAAAAAAAAA,2
472,325713bd-146b-4e0e-a288-27990b7e8111,polyA,AAAAAAAAAAA,+,394,404,CCAAAAAAAAA,4
494,325713bd-146b-4e0e-a288-27990b7e8111,polyT,TTTTTTTTTTT,-,415,425,GGTTTTTTTTT,4
516,325713bd-146b-4e0e-a288-27990b7e8111,10x_left_adapter,CTACACGACGCTCTTCCGATCT,-,450,471,CTACACGACGCTCTTCCGATCT,4


In [7]:
get_list = lambda x: sorted(list(set(x)))

gx = df.groupby('seqID').agg(
    total_matches = ('patternName', 'nunique'),
    matches = ('patternName', get_list)
).reset_index()

gx['match_coverage'] = gx['matches'].apply(len)

mlb = MultiLabelBinarizer()
ont_hot = mlb.fit_transform(gx['matches'])
ont_hot = pd.DataFrame(ont_hot, columns=mlb.classes_)

gx = pd.concat([gx, ont_hot], axis=1)
gx.head()

,seqID,total_matches,matches,match_coverage,10x_left_adapter,10x_right_adapter,biolegend_1,biolegend_2,biolegend_3,biosig,polyA,polyT
0,00000ed6-7b16-499d-bae7-66ce82fac6f9,4,"[10x_left_adapter, 10x_right_adapter, polyA, p...",4,1,1,0,0,0,0,1,1
1,00001398-f4f6-4c46-a66a-0db0fdf8ae6e,4,"[10x_left_adapter, 10x_right_adapter, polyA, p...",4,1,1,0,0,0,0,1,1
2,00001973-4b47-46a3-b923-fc076d0bad99,2,"[10x_left_adapter, 10x_right_adapter]",2,1,1,0,0,0,0,0,0
3,00001e91-7321-4821-8b0b-8ccf0dc18ace,4,"[10x_left_adapter, 10x_right_adapter, polyA, p...",4,1,1,0,0,0,0,1,1
4,00002508-570d-41c9-810c-d35d717633a5,4,"[10x_left_adapter, 10x_right_adapter, polyA, p...",4,1,1,0,0,0,0,1,1


In [8]:
gx[['10x_left_adapter', '10x_right_adapter', 'biolegend_1', 'biolegend_2',
       'biolegend_3', 'biosig', 'polyA', 'polyT']].sum(axis=0) / gx.shape[0]


10x_left_adapter     0.657030
10x_right_adapter    0.827778
biolegend_1          0.000843
biolegend_2          0.003611
biolegend_3          0.002964
biosig               0.029666
polyA                0.928313
polyT                0.928313
dtype: float64

In [9]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

In [ ]:
mlb = MultiLabelBinarizer()
ont_hot = mlb.fit_transform(gx['matches'])
ont_hot = pd.DataFrame(ont_hot, columns=mlb.classes_)
ont_hot.head()
# gxt = pd.concat([gx, ont_hot], axis=0)
# gxt.head()